In [1]:
#imports
import torch
import numpy as np
import matplotlib.pyplot as plt
import random
import math

In [2]:
%run dataloader.ipynb

In [3]:
batch_size=500
train_dataloader = DataLoader(train_set, collate_fn=collate_fn, batch_size=batch_size, shuffle=True)
dev_dataloader = DataLoader(dev_set, collate_fn=collate_fn, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_set, collate_fn=collate_fn, batch_size=batch_size, shuffle=True)

In [4]:
gpu_id = 'cuda:3'
device = torch.device(gpu_id if torch.cuda.is_available() else 'cpu')

In [13]:
humor = 0
no_humor = 0
for batch_idx, batch in enumerate(dev_dataloader, 0):
    x_p,x_c,y,hid,x_p_len=map(lambda x: x.to(device), batch)
    for label in y:
        if label == 0:
            no_humor += 1
        if label == 1:
            humor += 1

print(humor)
print(no_humor)

/tmp/ipykernel_2071247/4016921915.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  context_w=np.array(self.language_sdk[hid]['context_embedding_indexes'])
/tmp/ipykernel_2071247/4016921915.py:103: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  context_of=np.array(self.word_aligned_openface_sdk[hid]['context_features'])
/tmp/ipykernel_2071247/4016921915.py:104: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you 

486
486


In [5]:
class AE(torch.nn.Module):
    def __init__(self):
        super().__init__()
          
        # Building an linear encoder with Linear
        # layer followed by Relu activation function
        # 81 ==> 8
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(81, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 8)
        )
          
        # Building an linear decoder with Linear
        # layer followed by Relu activation function
        # 8 ==> 81
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(8, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 81),
            torch.nn.Sigmoid()
        )
  
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [6]:
class AE(torch.nn.Module):
    def __init__(self):
        super().__init__()
          
        # Building an linear encoder with Linear
        # layer followed by Relu activation function
        # 81 ==> 8
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(752, 512),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(512, 256),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(256, 128)
        )
          
        # Building an linear decoder with Linear
        # layer followed by Relu activation function
        # 8 ==> 81
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(128, 256),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(256, 512),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(512, 752)
        )
  
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [11]:
# rnn
class RNN(torch.nn.Module):

    def __init__(self, input_dim, hidden_layer_size_1, hidden_layer_size_2) -> None:
        super().__init__()
        #self.rnn1_s0 = torch.nn.Parameter(torch.tensor(np.random.normal(0.0, 1.0, (hidden_layer_size_1,)), dtype=torch.float32))
        #self.rnn1_c0 = torch.nn.Parameter(torch.tensor(np.random.normal(0.0, 1.0, (hidden_layer_size_1,)), dtype=torch.float32))
        #self.rnn2_s0 = torch.nn.Parameter(torch.tensor(np.random.normal(0.0, 1.0, (hidden_layer_size_1,)), dtype=torch.float32))
        #self.rnn2_c0 = torch.nn.Parameter(torch.tensor(np.random.normal(0.0, 1.0, (hidden_layer_size_1,)), dtype=torch.float32))
        self.rnn = torch.nn.LSTM(input_dim, hidden_layer_size_1, batch_first=True, num_layers=2)
        self.dropout = torch.nn.Dropout(0.2)
        #self.w1 = torch.nn.Parameter(torch.tensor(np.random.normal(0.0, 1.0, (hidden_layer_size_1, hidden_layer_size_2)), dtype=torch.float32))
        #self.b1 = torch.nn.Parameter(torch.zeros((1,), dtype=torch.float32))
        self.linear = torch.nn.Linear(hidden_layer_size_1, 1)

    def forward(self, x, text_lens):
        batch_size = x.shape[0]
        '''
        s0 = torch.stack((
            self.rnn1_s0,
            self.rnn2_s0,
        ), dim=0)
        s0 = s0.unsqueeze(1).tile((1, batch_size, 1))

        c0 = torch.stack((
            self.rnn1_c0,
            self.rnn2_c0,
        ), dim=0)
        c0 = c0.unsqueeze(1).tile((1, batch_size, 1))
        '''

        interm_states, _ = self.rnn(x)
        final_state = interm_states[torch.arange(batch_size), text_lens - 1, :]
        #hidden = torch.nn.functional.leaky_relu(final_state@self.w1 + self.b1)
        h = self.dropout(final_state)
        return self.linear(final_state)

In [12]:
# input features
###
#idxs = torch.arange(300) # text, 300
#idxs = torch.arange(300, 381) # speech, 81
#idxs = torch.arange(381, 752) # visual, 371
#idxs = torch.arange(381) # text + speech, 381
#idxs = torch.cat((torch.arange(300), torch.arange(381, 752))) # text + visual, 671
#idxs = torch.arange(300, 752) # speech + visual, 452
idxs = torch.arange(752) # text + speech + visual, 752

ae = torch.load('autoencoder_all_2.pth')
ae.to(device)

device = torch.device(gpu_id if torch.cuda.is_available() else 'cpu')
rnn = RNN(input_dim=1024, hidden_layer_size_1=512, hidden_layer_size_2=128)
rnn.to(device)

optimiser = torch.optim.Adam(rnn.parameters(), lr=0.0001)
epochs = 100
patience = 20

print('step', 'error')
dev_accs = []
best_dev_acc = 0
errors = []
num_bad_epochs = 0
num_epochs = 0
best_epoch = 0
for epoch in range(1, epochs+1):
    for batch_idx, batch in enumerate(train_dataloader, 0):
        x_p,x_c,y,hid,x_p_len=map(lambda x: x.to(device), batch)
        x_p_transformed = []
        for sent in x_p:
            speech_feats = sent[:, idxs].detach().tolist()
            speech_feats = torch.tensor(speech_feats, dtype=torch.float32, device=device)
            with torch.no_grad():
                speech_feats_transformed, _ = ae(speech_feats)
            speech_feats_transformed = speech_feats_transformed.detach().tolist()
            x_p_transformed.append(speech_feats_transformed)
        x_p_transformed = torch.tensor(x_p_transformed, dtype=torch.float32, device=device)

        optimiser.zero_grad()
        output = rnn(x_p_transformed, x_p_len)
        error = torch.nn.functional.binary_cross_entropy_with_logits(output, y)
        error.backward()
        optimiser.step()
    
    print(epoch, error.detach().tolist())
    errors.append(error.detach().tolist())

    with torch.no_grad():
        matches = 0
        total = 0
        for batch_idx, batch in enumerate(dev_dataloader, 0): 
            x_p,x_c,y,hid,x_p_len=map(lambda x: x.to(device), batch)
            x_p_transformed = []
            for sent in x_p:
                speech_feats = sent[:, idxs].detach().tolist()
                speech_feats = torch.tensor(speech_feats, dtype=torch.float32, device=device)
                with torch.no_grad():
                    speech_feats_transformed, _ = ae(speech_feats)
                speech_feats_transformed = speech_feats_transformed.detach().tolist()
                x_p_transformed.append(speech_feats_transformed)
            x_p_transformed = torch.tensor(x_p_transformed, dtype=torch.float32, device=device)

            outputs = torch.sigmoid(rnn(x_p_transformed, x_p_len))
            predictions = (outputs > 0.5)*1
            matches += (y == predictions).sum()
            total += len(y)
        acc = matches / total
        dev_accs.append(float(acc))
        print('dev accuracy: {:.2%}'.format(float(acc)))

    if acc > best_dev_acc:
        best_dev_acc = acc
        best_epoch = epoch
        torch.save(rnn, 'punchline_rnn_text_speech_visual.pth')
    else:
        num_bad_epochs += 1
        if num_bad_epochs == patience:
            num_epochs = epoch
            break

plt.plot(range(1, num_epochs+1), dev_accs, 'b')
plt.title('dev set accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()

plt.plot(range(1, num_epochs+1), errors, 'r')
plt.title('train error')
plt.xlabel('epochs')
plt.ylabel('error')
plt.show()

print('best model after {} epochs with dev set accuracy {}'.format(best_epoch, best_dev_acc))
rnn = torch.load('punchline_rnn_text_speech_visual.pth')

step error


/tmp/ipykernel_3296201/4016921915.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  context_w=np.array(self.language_sdk[hid]['context_embedding_indexes'])
/tmp/ipykernel_3296201/4016921915.py:103: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  context_of=np.array(self.word_aligned_openface_sdk[hid]['context_features'])
/tmp/ipykernel_3296201/4016921915.py:104: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you 

1 0.7068066596984863
dev accuracy: 54.84%
2 0.6906037926673889
dev accuracy: 53.91%


KeyboardInterrupt: 

In [77]:
hid_humor_t = {}
hid_humor_f = {}
hid_nonhumor_t = {}
hid_nonhumor_f = {}

with torch.no_grad():
    matches = 0
    total = 0
    for batch_idx, batch in enumerate(test_dataloader, 0): 
        x_p,x_c,y,hid, x_p_len=map(lambda x: x.to(device), batch)
        outputs = torch.sigmoid(rnn(x_p[:, :, idxs], x_p_len))
        predictions = (outputs > 0.5)*1
        matches += (y == predictions).sum()
        total += len(y)
        for j, id in enumerate(hid.detach().tolist()):
            if predictions[j] == 1 and y[j] == 1:
                hid_humor_t[id] = outputs[j].detach().tolist()[0]
            elif predictions[j] == 1 and y[j] == 0:
                hid_humor_f[id] = outputs[j].detach().tolist()[0]
            elif predictions[j] == 0 and y[j] == 0:
                hid_nonhumor_t[id] = outputs[j].detach().tolist()[0]
            elif predictions[j] == 0 and y[j] == 1:
                hid_nonhumor_f[id] = outputs[j].detach().tolist()[0]
            else:
                print('something is wrong')
    acc = matches / total
    print('test accuracy: {:.2%}'.format(float(acc)))

/tmp/ipykernel_1818251/4016921915.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  context_w=np.array(self.language_sdk[hid]['context_embedding_indexes'])
/tmp/ipykernel_1818251/4016921915.py:103: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  context_of=np.array(self.word_aligned_openface_sdk[hid]['context_features'])
/tmp/ipykernel_1818251/4016921915.py:104: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you 

test accuracy: 63.67%


In [73]:
print(len(hid_humor_t))
print(len(hid_nonhumor_t))
print(len(hid_humor_f))
print(len(hid_nonhumor_f))

language_sdk=load_pickle("language_sdk.pkl")

sorted_humor_t = {k: v for k, v in sorted(hid_humor_t.items(), key=lambda item: item[1], reverse=True)}
sorted_nonhumor_t = {k: v for k, v in sorted(hid_nonhumor_t.items(), key=lambda item: item[1], reverse=True)}
sorted_humor_f = {k: v for k, v in sorted(hid_humor_f.items(), key=lambda item: item[1], reverse=True)}
sorted_nonhumor_f = {k: v for k, v in sorted(hid_nonhumor_f.items(), key=lambda item: item[1], reverse=True)}

for hid, output in list(sorted_humor_t.items())[:5]:
    print(language_sdk[hid]['punchline_sentence'])
print()
for hid, output in list(sorted_humor_f.items())[:5]:
    print(language_sdk[hid]['punchline_sentence'])
print()
for hid, output in list(sorted_nonhumor_t.items())[:5]:
    print(language_sdk[hid]['punchline_sentence'])
print()
for hid, output in list(sorted_nonhumor_f.items())[:5]:
    print(language_sdk[hid]['punchline_sentence'])

323
270
220
167
the good news is i married her the next year
i'm not going to talk about that tonight
i haven't come to the best part
but i still have six and a half minutes
they probably should have marketed this as some of the guardians of the galaxy

i said i'd like to be a writer
another called twister and so on and so on
i would have told you myself that i was the last person on earth who would stay with a man who beats me but in fact i was a very typical victim because of my age
oh let me kind of tell you a funny little play bit
you can experience their sense of place

and you all have chosen your own instruments to fulfill this mission of creating a better world
and the satellite imagery we have of it today is old
and as you're thinking about this incredible ability to make life do what you want it to do what it's programmed to do what you end up doing is taking what we've been doing for thousands of years which is breeding changing mixing matching all kinds of life forms and we